In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

#output file path for data resource files and figures
outputFilePath = os.path.join("Output")

In [39]:
#read the weather_cities.csv into a dataframe
weather_cities_df = pd.read_csv(os.path.join("Output","city_weather.csv"),delimiter=",")
weather_cities_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,5882953,Aklavik,CA,68.2191,-135.0107,-20.20,69,75,3.44
1,909887,Luangwa,ZM,-15.6167,30.4167,91.69,22,35,5.26
2,4023117,Dolores Hidalgo Cuna de la Independencia Nacional,MX,21.1561,-100.9308,60.87,34,48,3.44
3,2170430,Cootamundra,AU,-34.6500,148.0333,53.94,72,100,18.34
4,2037485,Erenhot,CN,43.6475,111.9767,22.95,57,0,4.21
...,...,...,...,...,...,...,...,...,...
567,638936,Rovaniemi,FI,66.5000,25.7167,-4.59,84,75,3.44
568,1655140,Pakxan,LA,18.3778,103.6601,57.22,66,41,3.85
569,2426370,Umm Hājar,TD,13.2954,19.6966,86.88,18,69,4.97
570,1519385,Sergeevka,KZ,53.8800,67.4158,25.36,90,81,11.90


In [41]:
#humidity heatmap

#store locations by latitude and longitude
locations = weather_cities_df[["Lat","Lng"]].astype(float)

#drop na values (though these should have been handled in WeatherPy before saving to csv)
#do not rid of the index though - indexes will correspond between locations and humididty
weather_cities_df = weather_cities_df.dropna()

#convert humidity to float
humidity = weather_cities_df["Humidity"].astype(float)
maxHumidity = humidity.max()
#create a humidity heatmap layer
fig = gmaps.figure()

heatLayer_humidity = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity = maxHumidity, point_radius = 3)

fig.add_layer(heatLayer_humidity)

fig


Figure(layout=FigureLayout(height='420px'))

In [32]:
# Create new DataFrame using new criteria 

narrowed_city_df = weather_cities_df.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Temperature'] >= 70]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Temperature'] <= 80]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Clouds'] == 0]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df


,index,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,72,3896924,Cañete,CL,-37.8000,-73.4000,75.79,38,0,9.08
1,232,935214,Saint-Pierre,RE,-21.3393,55.4781,77.68,76,0,3.44
2,285,292968,Abu Dhabi,AE,24.4667,54.3667,74.57,64,0,5.75
3,335,286621,Salalah,OM,17.0151,54.0924,74.97,51,0,1.90
4,460,3531368,Celestún,MX,20.8667,-90.4000,79.70,50,0,4.32
5,547,3868707,Valdivia,CL,-39.8142,-73.2459,71.69,26,0,7.96
6,558,1067565,Beloha,MG,-25.1667,45.0500,79.18,63,0,8.03


In [33]:
# Hotel Mapping

#create dataframe for the hotel information
hotel_df = narrowed_city_df.loc[:,["Lat","Lng","City","Country"]]
# hotel_df = pd.DataFrame(hotel_df)
hotel_df

,Lat,Lng,City,Country
0,-37.8000,-73.4000,Cañete,CL
1,-21.3393,55.4781,Saint-Pierre,RE
2,24.4667,54.3667,Abu Dhabi,AE
3,17.0151,54.0924,Salalah,OM
4,20.8667,-90.4000,Celestún,MX
5,-39.8142,-73.2459,Valdivia,CL
6,-25.1667,45.0500,Beloha,MG


In [34]:
#set up lists for hotel information
hotelName = []
hotelAddress = []
hotelRating = []
hotelLat = []
hotelLng = []

for index, row in hotel_df.iterrows():
    #current info
    currentCity = row["City"]
    currentCountry = row["Country"]
    
    #geocoordinates
    target_coordinates = f"{row['Lat']},{row['Lng']}"
    target_radius = 5000 #this is in meters
    target_type = "hotel"
    target_keyword = "hotel"

    #set parameters
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "keyword": target_keyword,
        "key": g_key
    }
    
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #run request using params dictionary
    response = requests.get(base_url,params=params).json()
    #simplify finding results
    results = response['results']
     
    try:
        hotelName.append(results[0]['name'])
        hotelAddress.append(results[0]['vicinity'])
        hotelRating.append(results[0]['rating'])
        hotelLat.append(results[0]['geometry']['location']['lat'])
        hotelLng.append(results[0]['geometry']['location']['lng'])
        print(f"Processeing Record for a {target_keyword} in {currentCity}, {currentCountry}")
    except:
        hotelName.append(np.nan)
        hotelAddress.append(np.nan)
        hotelRating.append(np.nan)
        hotelLat.append(np.nan)
        hotelLng.append(np.nan)
        print(f"No results/{target_keyword} found in {currentCity}, {currentCountry}. Skipping...")   

Processeing Record for a hotel in Cañete, CL
Processeing Record for a hotel in Saint-Pierre, RE
Processeing Record for a hotel in Abu Dhabi, AE
Processeing Record for a hotel in Salalah, OM
Processeing Record for a hotel in Celestún, MX
Processeing Record for a hotel in Valdivia, CL
No results/hotel found in Beloha, MG. Skipping...


In [35]:
#add new information into the cities_df
hotel_df["Hotel Name"] = hotelName
hotel_df["Hotel Address"] = hotelAddress
hotel_df["Hotel Rating"] = hotelRating
hotel_df["Hotel Latitude"] = hotelLat
hotel_df["Hotel Longitude"] = hotelLng

hotel_df

#remove NaN rows
#did not drop indexes for locations without hotel found - can see how many hotels are found for top locations
hotelFound_df = hotel_df.dropna()
hotelFound_df

,Lat,Lng,City,Country,Hotel Name,Hotel Address,Hotel Rating,Hotel Latitude,Hotel Longitude
0,-37.8000,-73.4000,Cañete,CL,Albergo Italiano,"Villagrán 1168, Canete",4.4,-37.797160,-73.397158
1,-21.3393,55.4781,Saint-Pierre,RE,Hôtel Le Saint-Pierre,"51 Av. des Indes, Saint-Pierre",4.3,-21.341364,55.474339
2,24.4667,54.3667,Abu Dhabi,AE,Emirates Palace,W Corniche Rd,4.8,24.461644,54.317283
3,17.0151,54.0924,Salalah,OM,Salalah Gardens Hotel,Al Rabat Street، Salalah,4.8,17.022848,54.064693
4,20.8667,-90.4000,Celestún,MX,Hotel Gutierrez,"Calle 12 x 13 y 15 #104, Centro, Celestún",4.4,20.860191,-90.400895
5,-39.8142,-73.2459,Valdivia,CL,Hotel Villa del Río,"España 1025, Valdivia",4.4,-39.807864,-73.235582


In [36]:
print(f"There were {len(hotel_df)} locations matching vacation requirements and {len(hotelFound_df)} hotels found.")

There were 7 locations matching vacation requirements and 6 hotels found.


In [37]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotelFound_info = [info_box_template.format(**row) for index, row in hotelFound_df.iterrows()]
locations = hotelFound_df[["Hotel Latitude", "Hotel Longitude"]]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotelFound_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))